# Initialization

In [0]:
# Modules
import numpy as np

In [0]:
# Dynamic params intialization of Weights abd Bias (First layer size depends on Input shape and Last layer size depends on Number_of_class)
def init_params(layer_dims,num_of_class,X):
  params = dict()
  layer_dims.insert(0,X.shape[0]) 
  
  for i in range(1,len(layer_dims)):
    params['W'+str(i)] = np.random.randn(layer_dims[i],layer_dims[i-1]) * 0.01
    params['b'+str(i)] = np.zeros((layer_dims[i],1))
    #print(params['W'+str(i)].shape,params['b'+str(i)].shape)
  
  params['W'+str(i+1)] = np.random.randn(num_of_class,layer_dims[i]) * 0.01
  params['b'+str(i+1)] = np.zeros((num_of_class,1))
  #print(params['W'+str(i)].shape,params['b'+str(i)].shape)
  
  return params

# Forward Pass

In [0]:
# Few Activaion func for Forward pass
def sigmoid(Z):
  A = 1/(1+np.exp(-Z))
  return A

def relu(Z):
  A = np.maximum(0,Z)
  return A

In [0]:
def linear_forward(A,W,b,activation):
  Z = np.dot(W,A) + b
  if activation == 'sigmoid':
    A = sigmoid(Z)
  if activation == 'relu':
    A = relu(Z)
  return A,Z

In [0]:
def linear_forwardpass(params,X,activation):
  # Init of A and Z
  A = dict()
  Z = dict()
  A['A'+str(0)] = X
  for i in range(1,params//2):
    A['A'+str(i)],Z['Z'+str(i)] = linear_forward(A['A'+str(i-1)],params['W']+str(i),params['b'+str(i)],activation)
  AL,Z['Z'+str(i+1)] = linear_forward(A['A'+str(i)],params['W']+str(i+1),params['b'+str(i+1)],'sigmoid')
  return AL,Z,A

# Backward Pass

In [0]:
# Few Activaion func for Backward pass
def sigmoid_backward(dA,Z):
  s = 1/(1+np.exp(-Z))
  dZ = dA * s * (1-s)
  return dZ

def relu_backward(dA,Z):
  dZ = np.array(dA, copy=True) # just converting dz to a correct object.
  # When z <= 0, you should set dz to 0 as well. 
  dZ[Z <= 0] = 0
  return dZ
    

In [0]:
def liner_backward(dA,i,Z,A_prev,activation='sigmoid'):
    # dZ:
    if activation=='sigmoid':
      dZ = sigmoid_backward(dA,Z)
    if activation=='relu':
      dZ = relu_backward(dA,Z)

    m = A_prev.shape[1]
    dW = 1./m * np.dot(dZ,A_prev.T)
    db = 1./m * np.sum(dZ, axis = 1, keepdims = True)
    dA_prev = np.dot(W.T,dZ)

In [0]:
def linear_backwardpass(AL,Y,l,Z,A,activation): 
    # dZ depends on dA & Z (Using dA & Z - dZ is computed)
    # dW,db depends on dZ (Using dZ - dW and db is computed)
    grads = dict()
    dAL = -1* ( np.divide( Y,AL ) + np.divide( (1-Y),(1-AL) ) )
    #Last Layer alone
    grads['dA'+str(l)],grads['dW'+str(l+1)],grads['db'+str(l+1)] = liner_backward(dAL,l,Z['Z'+str(l+1)],A['A'+str(l)])  
    for i in range(l,0,-1):
      grads['dA'+str(i-1)],grads['dW'+str(i)],grads['db'+str(i)] = liner_backward(grads['dA'+str(i)],i,Z['Z'+str(i)],A['A'+str(l-1)],activation)    
    return grads    

# Cost per Epoch

In [0]:
def cost_func(Y,AL,X):
  curr_loss = ( -1/X.shape[1] ) * ( np.sum( np.multiply( Y,np.log(AL) ,axis=1) ) + np.sum(np.multiply( (1-Y),np.log(1-AL) ) ,axis=1) )
  return curr_loss

# Update Parameters

In [0]:
def update_params(params ,grads ,learning_rate):
  for i in range(1,(params)//2+1):
    params['W'+str(i)] = params['W'+str(i)] - learning_rate * grads['dW'+str(i)]
    params['b'+str(i)] = params['b'+str(i)] - learning_rate * grads['db'+str(i)]
  return params

# Dynamic Neural Network

In [0]:
# Main Neural network
def linear_model(X,Y,learning_rate,anneal,activation,num_of_class,layer_dims,epochs):
  # Init part 
  params = init_params(layer_dims,num_of_class,X)
  l = len(layer_dims)
  # costs over epochs
  costs = []
  # Epochs
  for i in range(epochs):
    # Forward Pass
    AL,Z,A = linear_forwardpass(params,X,activation)
    
    # cost at each epoch
    cost = cost_func(Y,AL,X)
    
    # Back Prop (Derivatives)
    grads = linear_backwardpass(AL,Y,l,Z,A,activation)
    
    # Optimizer
    # General Gradient descent

    # Update params
    params = update_params(params ,grads ,learning_rate)
    
    # anneal (Add)
    if anneal == True:
      learning_rate /= 2

    # Loss over every 100 epochs
    if i > 0  and i%100 == 0:
      print('Cost at epoch is : {}'.format(cost))
      costs.append(cost)